In [64]:
!pip install --upgrade gspread
!pip install -U gspread-dataframe

In [100]:
import pandas as pd
import numpy as np
from google.colab import auth
auth.authenticate_user()
pd.set_option("display.max_rows", None)
import gspread
from oauth2client.client import GoogleCredentials
import pandas as pd
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [101]:
wb = gc.open_by_key('1-txdg3xqfS0PQg0m5i0VmNiUwHwc7uhI3sTzGUge3Ok')
ws1 = wb.worksheet("address")
rows1 = ws1.get_all_values()
df1 = pd.DataFrame.from_records(rows1[1:], columns=rows1[0])

In [102]:
df1.head()

,business,storeid,coordinates
0,100houz,5f92cfb1bc5bab000668c924,"101.78968286023418,2.9808692932128906"
1,103coffeeworkshop,6017aa6ef3e0fb00060b2a60,"101.69389392979005,3.0709849395810926"
2,103coffeeworkshop,5df7487285ac1e35e7dffa93,"101.72741677073653,3.1252483192884744"
3,11atvista,566bc2780c267bf50332dae8,"101.52430979495061,3.071352005004883"
4,11atvista,5cd568836be1be3f082c658d,"101.48143533120928,3.062784194946289"


In [105]:
df1 = pd.concat([df1[['business','storeid','coordinates']], df1['coordinates'].str.split(',', expand=True)], axis=1)

In [106]:
df1.head()

,business,storeid,coordinates,0,1
0,100houz,5f92cfb1bc5bab000668c924,"101.78968286023418,2.9808692932128906",101.78968286023418,2.9808692932128906
1,103coffeeworkshop,6017aa6ef3e0fb00060b2a60,"101.69389392979005,3.0709849395810926",101.69389392979005,3.0709849395810926
2,103coffeeworkshop,5df7487285ac1e35e7dffa93,"101.72741677073653,3.1252483192884744",101.72741677073653,3.1252483192884744
3,11atvista,566bc2780c267bf50332dae8,"101.52430979495061,3.071352005004883",101.52430979495061,3.071352005004883
4,11atvista,5cd568836be1be3f082c658d,"101.48143533120928,3.062784194946289",101.48143533120928,3.062784194946289


In [107]:
df1.columns

Index(['business', 'storeid', 'coordinates', 0, 1], dtype='object')

In [108]:
df1 = df1.rename(columns={0: 'Long', 1: 'Lat'})

In [109]:
df1.columns

Index(['business', 'storeid', 'coordinates', 'Long', 'Lat'], dtype='object')

In [110]:
df1 = df1[['business', 'storeid', 'Lat', 'Long',]]

In [111]:
df1.head()

,business,storeid,Lat,Long
0,100houz,5f92cfb1bc5bab000668c924,2.9808692932128906,101.78968286023418
1,103coffeeworkshop,6017aa6ef3e0fb00060b2a60,3.0709849395810926,101.69389392979005
2,103coffeeworkshop,5df7487285ac1e35e7dffa93,3.1252483192884744,101.72741677073653
3,11atvista,566bc2780c267bf50332dae8,3.071352005004883,101.52430979495061
4,11atvista,5cd568836be1be3f082c658d,3.062784194946289,101.48143533120928


In [112]:
df1.shape

(6712, 4)

In [113]:
df1.isna().sum()

business       0
storeid        0
Lat         1338
Long           0
dtype: int64

In [115]:
df1[df1['Lat'].isna()]

,business,storeid,Lat,Long
8,12oclockbistro,5fa8b1701b872800069650a6,None,
16,1962,617a2a830643ea0006c4a75d,None,
35,3littlechef,611f4d1a86b4e300066ce2a0,None,
36,3littlechef,61232567e61e490006e6a0e8,None,
48,6yicafe,5e5f2daf739454621e81f2ff,None,


In [118]:
df1[df1['Long']== ''].head()

,business,storeid,Lat,Long
8,12oclockbistro,5fa8b1701b872800069650a6,None,
16,1962,617a2a830643ea0006c4a75d,None,
35,3littlechef,611f4d1a86b4e300066ce2a0,None,
36,3littlechef,61232567e61e490006e6a0e8,None,
48,6yicafe,5e5f2daf739454621e81f2ff,None,


In [119]:
cleaned = df1.dropna()

In [120]:
cleaned.shape, df1.shape

((5374, 4), (6712, 4))

In [ ]:
cleaned.head()

In [121]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="geoapiExercises")
Latitude = cleaned.Lat[0]
Longitude = cleaned.Long[0]
 
location = geolocator.reverse(Latitude+","+ Longitude)

In [122]:
location

Location(Taman Hijau, Majlis Perbandaran Kajang, Hulu Langat, Selangor, 43000, Malaysia, (2.9808702017208515, 101.78966346413253, 0.0))

In [98]:
location[0]

'Taman Hijau, Majlis Perbandaran Kajang, Hulu Langat, Selangor, 43000, Malaysia'

In [44]:
type(location)

geopy.location.Location

In [45]:
type(location[0])

str

In [126]:
test = cleaned[:10]

In [ ]:
test

In [142]:
z = []
for Lat,Long in zip(cleaned.Lat, cleaned.Long):
  z.append(Lat+','+ Long)

# z = []
# for Lat,Long in zip(test.Lat, test.Long):
#   z.append(Lat+','+ Long)

In [143]:
loc=[]
for x in z:
  loc.append(geolocator.reverse(x))


In [144]:
new_lis = []
for y in loc:
  new_lis.append(y[0])

In [145]:
new_df = pd.DataFrame(new_lis)

In [147]:
new_df.head()

,0
0,"Taman Hijau, Majlis Perbandaran Kajang, Hulu L..."
1,"Morning Market Sri Petaling, Jalan Radin Bagus..."
2,"AEON Taman Maluri Shopping Centre, Jalan Jejak..."
3,"Avisena Specialist Hospital, Jalan Ikhtisas, S..."
4,"Central i-City Shopping Centre, Persiaran Mult..."


In [148]:
cleaned.shape, new_df.shape

((5374, 4), (5374, 1))

In [161]:
cleaned = cleaned.reset_index()

In [162]:
cleaned = cleaned.drop(columns=['index'])

In [163]:
final_df = pd.concat([cleaned, new_df], axis=1)

In [164]:
final_df.head()

,business,storeid,Lat,Long,0
0,100houz,5f92cfb1bc5bab000668c924,2.9808692932128906,101.78968286023418,"Taman Hijau, Majlis Perbandaran Kajang, Hulu L..."
1,103coffeeworkshop,6017aa6ef3e0fb00060b2a60,3.0709849395810926,101.69389392979005,"Morning Market Sri Petaling, Jalan Radin Bagus..."
2,103coffeeworkshop,5df7487285ac1e35e7dffa93,3.1252483192884744,101.72741677073653,"AEON Taman Maluri Shopping Centre, Jalan Jejak..."
3,11atvista,566bc2780c267bf50332dae8,3.071352005004883,101.52430979495061,"Avisena Specialist Hospital, Jalan Ikhtisas, S..."
4,11atvista,5cd568836be1be3f082c658d,3.062784194946289,101.48143533120928,"Central i-City Shopping Centre, Persiaran Mult..."


In [166]:
final_df.to_csv('address-2021-2022-active-merchants.csv',index=False )

In [159]:
cleaned.shape

(5374, 4)

In [165]:
from gspread_dataframe import  set_with_dataframe
new_sh = wb.add_worksheet('address-2021-2022-active-merchants', rows=6000, cols=7)
set_with_dataframe(new_sh,final_df)

APIError: ignored